In [ ]:
import pandas as pd


In [ ]:

file_bank = "Cop1128/Copbank.xls" #Create the Bank frame
bank = pd.read_excel(file_bank)
#bank.set_index('Date', inplace=True)

file_xero = "Cop1128/Copxero.xls" #Create xero frame
xero = pd.read_excel(file_xero)
#xero.set_index('Date', inplace=True)  #the inplace makes sure the change remains intack

In [ ]:
#Creat lists of credit and Debit 
bankIncome = bank["Debit"].tolist()
bankExpense = bank["Credit"].tolist()
xeroIncome = xero["Debit"].tolist()
xeroExpense = xero["Credit"].tolist()

In [ ]:
#Clean The Lists
bankIncome1 = [x for x in bankIncome if str(x) != '-']
bankExpense1 = [x for x in bankExpense if str(x) != '-']
       
xeroIncome1 = [x for x in xeroIncome if str(x) != 'nan']
xeroExpense1 = [x for x in xeroExpense if str(x) != 'nan']


In [ ]:
def list_difference(b, a): # Removes the numbers from b for those that are found in a
    c = b.copy()  # or list(b)
    for item in a:
       try:
           c.remove(item)
       except ValueError:
           pass            #or maybe you want to keep a values here
    return c



def list_similar(b, a): # Removes the numbers from b for those that are found in a
    c = b.copy()  # or list(b)
    same = []
    for item in a:
       try:
        if item in c:
            same.append(item)
            c.remove(item)
       except ValueError:
           pass            #or maybe you want to keep a values here
    return same
 

In [ ]:
#differences between xero and bank  (values that dont unify postings and actual banks)

xero_unmatched_income = list_difference(xeroIncome1, bankIncome1)
one = pd.Series(xero_unmatched_income) #xero income not matching
xero_unmatched_expense = list_difference(xeroExpense1, bankExpense1)
two = pd.Series(xero_unmatched_expense)#xero expense not matching


bank_unmatched_income = list_difference(bankIncome1, xeroIncome1)
three = pd.Series(bank_unmatched_income) #bank income not matching
bank_unmatched_expense = list_difference(bankExpense1, xeroExpense1)
four = pd.Series(bank_unmatched_expense) #bank expense not matching

In [ ]:
#WRITE TO EXCEL THE DIFFERENCES
writer = pd.ExcelWriter('Cop1128/UnreconcilingAmounts.xlsx', engine='xlsxwriter')

one1 = xero[xero["Debit"].isin(one)]
one1.to_excel(writer, sheet_name="xeroIncome")

two1 = xero[xero["Credit"].isin(two)]
two1.to_excel(writer, sheet_name="xeroExpense")

three1 = bank[bank["Debit"].isin(three)]
three1.to_excel(writer, sheet_name="bankIncome")

four1 = bank[bank["Credit"].isin(four)]
four1.to_excel(writer, sheet_name="bankExpense")

writer.close()

In [ ]:
writer1 = pd.ExcelWriter('Cop1128/ReconcilingAmounts.xlsx', engine='xlsxwriter')

#PRODUCE THE INCOME RECONCILIATION WHERE RECONCILIED
similarIncome = list_similar(xeroIncome1, bankIncome1)

bankIncomeFrame = bank[bank['Debit'].isin(similarIncome)]  #These are similar Income
xeroIncomeFrame = xero[xero['Debit'].isin(similarIncome)] 

'''#bankIncomeFrame['Debit'].astype(int)
#bankIncomeFrame['Debit'].astype(int)
#IncomeReconciliation = pd.merge(bankIncomeFrame, xeroIncomeFrame, on="Debit", how='outer')'''

xeroIncomeFrame.to_excel(writer1, sheet_name='xeroIncome')
bankIncomeFrame.to_excel(writer1, sheet_name='bankIncome')


#PRODUCE THE EXPENSE RECONCILIATION WHERE RECONCILIED

similarExpense = list_similar(xeroExpense1, bankExpense1)

bankExpenseFrame = bank[bank['Credit'].isin(similarExpense)]  #These are similar Expense frames from bank and xero
xeroExpenseFrame = xero[xero['Credit'].isin(similarExpense)] 

xeroExpenseFrame.to_excel(writer1, sheet_name='xeroExpense')
bankExpenseFrame.to_excel(writer1, sheet_name='bankExpense')

writer1.close()